In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 20)

/kaggle/input/janatahack-crosssell-prediction/sample_submission.csv
/kaggle/input/janatahack-crosssell-prediction/test.csv
/kaggle/input/janatahack-crosssell-prediction/train.csv


In [2]:
train = pd.read_csv('/kaggle/input/janatahack-crosssell-prediction/train.csv',index_col=0)
test = pd.read_csv('/kaggle/input/janatahack-crosssell-prediction/test.csv',index_col=0)
sample_submission = pd.read_csv('/kaggle/input/janatahack-crosssell-prediction/sample_submission.csv',index_col=0)

In [3]:
train[['Driving_License','Previously_Insured','Policy_Sales_Channel']] = train[['Driving_License','Previously_Insured','Policy_Sales_Channel']].astype('object')
train['Response'] = train['Response'].astype('object')

test[['Driving_License','Previously_Insured','Policy_Sales_Channel']] = test[['Driving_License','Previously_Insured','Policy_Sales_Channel']].astype('object')

In [4]:
X = train.drop('Response',axis=1)
y = train['Response'].values

In [5]:
X = pd.get_dummies(X,drop_first=True)
test = pd.get_dummies(test,drop_first=True)

In [6]:
le = LabelEncoder()
y = le.fit_transform(y)

In [15]:
clf = LGBMClassifier(n_estimators=550,
                     learning_rate=0.03,
                     min_child_samples=40,
                     random_state=1,
                     colsample_bytree=0.5,
                     reg_alpha=2,
                     reg_lambda=2)

clf.fit(X, y, verbose=50,eval_metric = 'auc')

LGBMClassifier(colsample_bytree=0.5, learning_rate=0.03, min_child_samples=40,
               n_estimators=550, random_state=1, reg_alpha=2, reg_lambda=2)

In [19]:
lgb_pred = clf.predict_proba(X)[:,1]
roc_auc_score(y,lgb_pred)

0.8648823717319468

In [21]:
print(test.shape,X.shape)

missing_cols = set(X.columns) - set(test.columns)
for c in missing_cols:
    test[c] = 0

print(test.shape,X.shape)

#keeping the order of columns same for X and X_test
test = test[X.columns]

(127037, 154) (381109, 164)
(127037, 166) (381109, 164)


In [25]:
test_pred = clf.predict_proba(test)[:,1]

In [26]:
sample_submission['Response'] = test_pred

In [27]:
sample_submission.to_csv('Submission_v1.csv')

,Response
id,
381110,0.000556
381111,0.321699
381112,0.303322
381113,0.004241
381114,0.000524
...,...
508142,0.000365
508143,0.356520
508144,0.000323


In [42]:
# from sklearn.model_selection import StratifiedKFold
# from lightgbm import LGBMClassifier
# scores = []
# model = LGBMClassifier()
# skf = StratifiedKFold(n_splits=10, random_state=2020, shuffle=False)
# for train_index, val_index in skf.split(X, y):
#     print("TRAIN:", train_index, "val:", val_index)
#     X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#     y_train, y_val = y[train_index], y[val_index]
    
#     model.fit(X_train, y_train)

TRAIN: [ 38076  38077  38078 ... 381106 381107 381108] val: [    0     1     2 ... 38342 38343 38345]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
scores